<a href="https://colab.research.google.com/github/harenderk62/Delhi_House_Price_ML/blob/main/Delhi_House_Rent_Price_Data(web_scrapping).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


     |████████████████████████████████| 904 kB 15.1 MB/s 
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Rel

In [ ]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import re
import pandas as pd
import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from lxml import etree
from collections import OrderedDict
import time
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
link_lis = []

with requests.Session() as session:
    page_number = 1
    url = 'https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-'
    
    while True:
      print("Processing page: #{page_number}; url: {url}".format(page_number=page_number, url=url))
      response = session.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')

      string_res = soup.text
      lis_str = string_res.split(',')
      # lis_str
      lis_loop = [ x.replace('"url":"','').replace('"','')  for x in lis_str if '"url":"https://www' in x ]
      link_lis.extend(lis_loop)

      # check if there is next page, break if not
      next_link = soup.find("a", text="Next Page >")
      if next_link is None:
          break

      url = urljoin(url, next_link["href"])
      page_number += 1

data = pd.DataFrame(link_lis)
data.to_csv("all_links.csv",index=False)
print("Done.")

Processing page: #1; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-
Processing page: #2; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-2
Processing page: #3; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-3
Processing page: #4; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-4
Processing page: #5; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-5
Processing page: #6; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-6
Processing page: #7; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-7
Processing page: #8; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-8
Processing page: #9; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-9
Processing page: #10; url: https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid-page-10
Processing page: #11; url: https://www.99acres.com/property-for-rent-

In [ ]:
link_lis

['https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid',
 'https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-dda-santushti-apartments-vasant-kunj-south-delhi-950-sq-ft-spid-A58191098',
 'https://www.99acres.com/2-bhk-bedroom-independent-builder-floor-for-rent-in-sector-5-vaishali-ghaziabad-950-sq-ft-spid-M58306828',
 'https://www.99acres.com/3-bhk-bedroom-independent-builder-floor-for-rent-in-defence-colony-south-delhi-1500-sq-ft-spid-M57783976',
 'https://www.99acres.com/4-bhk-bedroom-independent-builder-floor-for-rent-in-ultra-luxurious-floors-vasant-vihar-vasant-vihar-south-delhi-2200-sq-ft-spid-K58364160',
 'https://www.99acres.com/studio-apartment-flat-for-rent-in-dlf-capital-greens-2-karampura-west-delhi-361-sq-ft-r4-spid-D52683604',
 'https://www.99acres.com/9-bhk-bedroom-independent-house-villa-for-rent-in-geetanjali-enclave-south-delhi-5400-sq-ft-r1-spid-N51145944',
 'https://www.99acres.com/4-bhk-bedroom-independent-builder-floor-for-rent-in-greater-kail

In [ ]:
len(link_lis)

46103

In [ ]:
def area(soup):
  if soup.find(id="superbuiltupArea_span"):
    return soup.find(id="superbuiltupArea_span").text

  elif soup.find(id="builtupArea_span"):
    return soup.find(id="builtupArea_span").text

  elif soup.find(id="carpetArea_span"):
    return soup.find(id="carpetArea_span").text

  elif soup.find(id="superArea_span"):
    return soup.find(id="superArea_span").text

  else:
    return 0


temp_lis = []

options = Options()
options.add_argument("--headless") # Runs Chrome in headless mode.
options.add_argument('--no-sandbox') # # Bypass OS security model
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")
driver = webdriver.Chrome(chrome_options=options, executable_path='chromedriver')
count =0
for link in link_lis:
  data_dict = {}
  if count % 10 == 0:
    print(link)
  count+=1
  try:
    driver.get(link)
    html = driver.page_source
    soup = BeautifulSoup(html)
    dom = etree.HTML(str(soup))
    try:
      data_dict['Title'] =  soup.find('title').text
    except:
      pass
    try:
      data_dict['bedRoomNum'] =  soup.find(id="bedRoomNum").text
    except:
      pass
    try:
      data_dict['bathroomNum'] =  soup.find(id="bathroomNum").text
    except:
      pass
    try:
      data_dict['balconyNum'] =  soup.find(id="balconyNum").text
    except:
      pass
    try:
      data_dict['additionalRooms'] =  soup.find(id="additionalRooms").text
    except:
      pass
    try:
      data_dict['pdPrice2'] =  soup.find(id="pdPrice2").text
      if not find(id="pdPrice2").text:
        data_dict['pdPrice2'] =  dom.xpath("/html/body/div[1]/div/div[2]/div[2]/div[1]/div/div/table/tr[1]/td[2]/div[2]/span").text
    except:
      pass
    try:
      
      data_dict['superbuiltupArea_span'] = area(soup)
      
    except Exception as e:
      print("fun error",e)
      pass
    try:
      data_dict['address'] =  soup.find(id="address").text
    except:
      pass
    try:
      data_dict['furnishing'] =  soup.find(id="furnishingLabel").text
    except:
      pass
    try:
      data_dict['available_for'] =  soup.find(id="availableForLabel").text
    except:
      pass
    try:
      data_dict['available_from'] =  dom.xpath("/html/body/div[1]/div/div[2]/div[2]/div[1]/div/div/table/tr[4]/td[1]/div[2]").text
    except:
      pass
    try:
      data_dict['floor_num'] =  soup.find(id="Floor_Num_Label").text
    except:
      pass
    try:
      data_dict['parking'] =  soup.find(id="Reserved_Parking_Label").text
    except:
      pass
    try:
      data_dict['electricityWaterCharges'] =  soup.find(id="electricityWaterCharges").text
    except:
      pass
    try:
      data_dict['Powerbackup_Label'] =  soup.find(id="Powerbackup_Label").text
    except:
      pass
    try:
      data_dict['Prop_Id'] =  soup.find(id="Prop_Id").text
    except:
      pass
    try:
      data_dict['Age_Label'] =  soup.find(id="Age_Label").text
    except:
      pass
    temp_lis.append(data_dict)
  except:
    continue
final = pd.DataFrame(temp_lis)
final.to_csv('delhi_house_rent.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: use options instead of chrome_options


https://www.99acres.com/property-for-rent-in-delhi-ncr-ffid
https://www.99acres.com/3-bhk-bedroom-independent-builder-floor-for-rent-in-lokvihar-apartment-pitampura-north-delhi-1620-sq-ft-spid-B58621248
https://www.99acres.com/3-bhk-bedroom-independent-builder-floor-for-rent-in-greater-kailash-2-south-delhi-2250-sq-ft-spid-T57799844
https://www.99acres.com/studio-apartment-flat-for-rent-in-dlf-capital-greens-2-karampura-west-delhi-361-sq-ft-r4-spid-D52683604
https://www.99acres.com/4-bhk-bedroom-independent-house-villa-for-rent-in-jaypee-greens-greater-noida-4500-sq-ft-spid-P58704322
https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-rent-in-dda-flats-sarita-vihar-sarita-vihar-south-delhi-2000-sq-ft-spid-N57868714
https://www.99acres.com/7-bhk-bedroom-independent-house-villa-for-rent-in-chanakya-puri-south-delhi-12000-sq-ft-spid-R58445542
https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-rent-in-sri-vinayak-apartments-sector-22-dwarka-delhi-1800-sq-ft-spid-O58475168
https: